In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [993 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,235 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [966 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,257 kB]
Hit:13 https://ppa.launchpadcontent.net/g

In [7]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [14]:
# 2. Create a temporary view of the DataFrame.
# 1. Read in the AWS S3 bucket into a DataFrame.
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
import time

# The URL of the CSV file on AWS S3
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add the file to Spark and read it
spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()

# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales_view")




+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [18]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

# Use Spark SQL to perform the query
result = spark.sql("""
    SELECT date_built AS YearBuilt, ROUND(AVG(price), 2) AS AvgPrice
    FROM home_sales_view
    WHERE bedrooms = 4
    GROUP BY date_built
    ORDER BY date_built
""")

# Show the result
result.show()





+---------+---------+
|YearBuilt| AvgPrice|
+---------+---------+
|     2010|296800.75|
|     2011| 302141.9|
|     2012|298233.42|
|     2013|299999.39|
|     2014|299073.89|
|     2015|307908.86|
|     2016|296050.24|
|     2017|296576.69|
+---------+---------+



In [19]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# Use Spark SQL to perform the query
result = spark.sql("""
    SELECT date_built AS YearBuilt, ROUND(AVG(price), 2) AS AvgPrice
    FROM home_sales_view
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""")

# Show the result
result.show()





+---------+---------+
|YearBuilt| AvgPrice|
+---------+---------+
|     2010|292859.62|
|     2011|291117.47|
|     2012|293683.19|
|     2013|295962.27|
|     2014|290852.27|
|     2015| 288770.3|
|     2016|290555.07|
|     2017|292676.79|
+---------+---------+



In [20]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
# Use Spark SQL to perform the query
result = spark.sql("""
    SELECT date_built AS YearBuilt, ROUND(AVG(price), 2) AS AvgPrice
    FROM home_sales_view
    WHERE bedrooms = 3
      AND bathrooms = 3
      AND floors = 2
      AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""")

# Show the result
result.show()



+---------+---------+
|YearBuilt| AvgPrice|
+---------+---------+
|     2010|285010.22|
|     2011|276553.81|
|     2012|307539.97|
|     2013|303676.79|
|     2014|298264.72|
|     2015|297609.97|
|     2016| 293965.1|
|     2017|280317.58|
+---------+---------+



In [21]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# Calculate the "view" rating for the average price of homes meeting the specified criteria

# Start measuring the time
start_time = time.time()

# Use Spark SQL to perform the query
result = spark.sql("""
    SELECT ROUND(AVG(view), 2) AS AvgViewRating
    FROM home_sales_view
    WHERE price >= 350000
""")

# Show the result
result.show()

# Calculate and print the runtime
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|AvgViewRating|
+-------------+
|        32.26|
+-------------+

--- 1.175097942352295 seconds ---
--- 6.556510925292969e-05 seconds ---


In [22]:
# 7. Cache the the temporary table home_sales.
# Cache the temporary table home_sales
df.cache()


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [26]:
# 8. Check if the table is cached.
# Check if the table home_sales_view is cached
is_cached = spark.catalog.isCached('home_sales_view')

if is_cached:
    print("The table home_sales_view is cached.")
else:
    print("The table home_sales_view is not cached.")


The table home_sales_view is cached.


In [27]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
# Filter out view ratings with an average price greater than or equal to $350,000 using the cached data

# Start measuring the time
start_time = time.time()

# Filter the cached DataFrame
filtered_df = df.filter(df['price'] >= 350000)

# Calculate the average view rating for the filtered data
result = filtered_df.agg({"view": "avg"})

# Show the result
result.show()

# Calculate and print the runtime
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))


+---------+
|avg(view)|
+---------+
|   32.264|
+---------+

--- 1.9181902408599854 seconds ---


In [39]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Partition the DataFrame by the "date_built" field and save it in Parquet format

# Define the output directory for the partitioned data (Windows format)
output_dir = "file:///C:/Users/gfran/Desktop/Homework2/Starter_Code/parquet_partitioned_data"

# Save the DataFrame in Parquet format with partitioning
df.write.partitionBy("date_built").parquet(output_dir)

# Replace "file:///C:/Users/gfran/Desktop/Homework2/Starter_Code/parquet_partitioned_data" with the actual path where you want to save the partitioned data.




In [42]:
# 11. Read the parquet formatted data.

# Define the path to the Parquet data directory
parquet_data_dir = "file:///C:/Users/gfran/Desktop/Homework2/Starter_Code/parquet_partitioned_data"  # Replace with your actual Parquet data directory path

# Read the Parquet data into a DataFrame
parquet_df = spark.read.parquet(parquet_data_dir)

# Now, you can use the 'parquet_df' DataFrame for further processing.



In [43]:
# 12. Create a temporary table for the parquet data.

# Create a temporary view for the Parquet data
parquet_df.createOrReplaceTempView("parquet_home_sales_view")

# Now, you can use the "parquet_home_sales_view" temporary table for Spark SQL queries.


In [46]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

# Filter out view ratings with an average price greater than or equal to $350,000 using the Parquet DataFrame
# Import necessary libraries
import time
from pyspark.sql.functions import round, avg

# Filter out view ratings with an average price greater than or equal to $350,000 using the Parquet DataFrame

# Start measuring the time
start_time = time.time()

# Filter the Parquet DataFrame
filtered_parquet_df = parquet_df.filter(parquet_df['price'] >= 350000)

# Calculate the average view rating for the filtered data, rounding to two decimal places
result = filtered_parquet_df.agg(round(avg("view"), 2).alias("avg_view"))

# Show the result
result.show()

# Calculate and print the runtime
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))






+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 1.4492034912109375 seconds ---


In [49]:
# Check if the table exists
if spark.catalog.tableExists("home_sales"):
    # Uncache the "home_sales" temporary table
    spark.catalog.uncacheTable("home_sales")
else:
    print("The 'home_sales' table does not exist.")


The 'home_sales' table does not exist.


In [48]:
# 14. Uncache the home_sales temporary table.
# Uncache the "home_sales" temporary table
spark.catalog.uncacheTable("home_sales")


AnalysisException: ignored

In [50]:
# 15. Check if the home_sales is no longer cached
# Check if the table exists
if spark.catalog.tableExists("home_sales"):
    # Uncache the "home_sales" temporary table
    spark.catalog.uncacheTable("home_sales")
else:
    print("The 'home_sales' table does not exist.")



The 'home_sales' table does not exist.
